In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

df=pd.read_csv(r'./data/clustered_data.csv')

In [27]:
df

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,...,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth,cluster
0,65.0,2,0,0,0,58138.0,1617.0,4905.0,58.0,635.0,...,120.5,81.0,88.0,8,10,4,3,0,7,0
1,68.0,2,0,1,2,46344.0,27.0,4355.0,38.0,11.0,...,2.0,1.0,6.0,1,1,2,2,0,5,1
2,57.0,2,1,0,0,71613.0,776.0,4554.0,26.0,426.0,...,111.0,21.0,42.0,8,2,10,1,0,4,0
3,38.0,2,1,1,1,26646.0,53.0,4381.0,26.0,11.0,...,10.0,3.0,5.0,2,0,4,2,0,6,1
4,41.0,4,1,1,1,58293.0,422.0,4403.0,94.0,173.0,...,46.0,27.0,15.0,5,3,6,5,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,55.0,2,1,1,1,61223.0,1341.0,4623.0,46.0,709.0,...,42.0,81.0,126.5,9,3,4,2,0,5,2
2236,76.0,4,1,1,3,64014.0,444.0,4261.0,56.0,406.0,...,0.0,0.0,8.0,8,2,5,7,1,7,2
2237,41.0,2,0,0,0,56981.0,1241.0,4397.0,91.0,908.0,...,32.0,12.0,24.0,2,3,13,1,1,6,0
2238,66.0,3,1,1,1,69245.0,843.0,4398.0,8.0,428.0,...,80.0,30.0,61.0,6,5,10,2,0,3,0


In [28]:
df.shape

(2240, 22)

In [29]:
X=df.drop('cluster',axis=1)
y=df['cluster']

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier


In [31]:
from catboost import CatBoostClassifier


models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'AdaBoost':AdaBoostClassifier(),
    'SVC':SVC(),
    'GradientBoost':GradientBoostingClassifier(),
    #'CatBoost':CatBoostClassifier(),
    'Logistic Regression':LogisticRegression(),
    'KNeighbors Classifier':KNeighborsClassifier()
}

In [32]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    

    models_list = []
    scores = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_pred = model.predict(X_test)

        score = accuracy_score(y_test,y_pred)
        
        model_name = list(models.keys())[i]
        print(f'---- score for --- {model_name} ----')
        print(f"{score}")
        models_list.append(model_name)
        scores.append(score)
    
    print()
    
    report = pd.DataFrame()
    report['Model_name'] = models_list
    report['Score'] = scores        
    return report

In [33]:
report = evaluate_models(X, y, models)

---- score for --- Random Forest ----
0.9575892857142857
---- score for --- Decision Tree ----
0.9375
---- score for --- AdaBoost ----
0.9441964285714286
---- score for --- SVC ----
0.7299107142857143
---- score for --- GradientBoost ----
0.9732142857142857
---- score for --- Logistic Regression ----
0.8772321428571429
---- score for --- KNeighbors Classifier ----
0.8102678571428571



In [34]:
report.sort_values(by='Score',ascending=False)

,Model_name,Score
4,GradientBoost,0.973214
0,Random Forest,0.957589
2,AdaBoost,0.944196
1,Decision Tree,0.937500
5,Logistic Regression,0.877232
6,KNeighbors Classifier,0.810268
3,SVC,0.729911


**Report** 

Gradient boosting gave the best accuracy

In [35]:
# Separate data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

X_train

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,...,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
994,68.0,2,0,1,2,64587.0,108.0,4249.0,49.0,66.0,...,16.0,0.0,6.0,20.0,1,1,4,2,0,3
2162,65.0,2,1,1,1,47320.0,414.0,4574.0,10.0,200.0,...,111.0,50.0,15.0,19.0,5,1,8,6,0,6
906,61.0,2,0,0,0,86429.0,1449.0,4462.0,10.0,464.0,...,556.0,29.0,18.0,37.0,7,4,7,0,1,2
572,44.0,1,0,1,1,38593.0,177.0,4531.0,42.0,51.0,...,49.0,17.0,24.0,24.0,4,1,3,3,0,8
1877,64.0,2,1,0,0,72905.0,1515.0,4401.0,52.0,407.0,...,445.0,120.5,81.0,126.5,3,7,9,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,53.0,2,1,1,2,44078.0,41.0,4252.0,17.0,24.0,...,10.0,2.0,0.0,4.0,2,0,3,2,0,5
1095,43.0,2,0,1,1,61825.0,424.0,4568.0,56.0,162.0,...,100.0,55.0,30.0,27.0,4,2,8,1,0,4
1130,71.0,3,1,1,1,67381.0,957.0,4772.0,67.0,815.0,...,53.0,11.0,0.0,70.0,2,2,9,4,1,7
1294,59.0,4,0,1,2,48918.0,62.0,4320.0,21.0,52.0,...,9.0,0.0,0.0,1.0,1,0,4,2,0,4


In [36]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

gb = GradientBoostingClassifier(random_state=42)

params = {
    'n_estimators': np.arange(100, 500, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'loss': ['log_loss', 'exponential'],
    'criterion': ['friedman_mse', 'squared_error']
}

ans = GridSearchCV(gb, params, cv=10, n_jobs=-1)
ans.fit(X_train, y_train)

print("Best Parameters:", ans.best_params_)
print("Best CV Score:", ans.best_score_)


Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.05, 'loss': 'log_loss', 'n_estimators': np.int64(300)}
Best CV Score: 0.9649232402417116


In [43]:
best_model=GradientBoostingClassifier(criterion='friedman_mse',learning_rate=0.05,loss='log_loss',n_estimators=300)

In [44]:
clf=best_model.fit(X_train,y_train)


In [45]:
y_pred=clf.predict(X_test)

In [46]:
accuracy_score(y_test,y_pred)

0.9613095238095238

In [47]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       179
           1       0.99      0.98      0.98       259
           2       0.93      0.96      0.95       234

    accuracy                           0.96       672
   macro avg       0.96      0.96      0.96       672
weighted avg       0.96      0.96      0.96       672



In [48]:
confusion_matrix(y_test,y_pred)

array([[168,   1,  10],
       [  0, 253,   6],
       [  7,   2, 225]])

- **Reports**

**We can see, that the model performed pretty well.**
- we have used Gradient Boosting classifier as it performed well than other models
- We got a good accuracy while predicting the test dataset.